## import modules

In [1]:
import cobra
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model
from cobra import Model, Reaction, Metabolite
from metnet import pathway
from cobra.flux_analysis import pfba
from cobra.flux_analysis.loopless import add_loopless, loopless_solution

## modify the strain

In [2]:
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    model = cobra.io.read_sbml_model('./iML1515.xml')
    
    ## add new metabolites
    tyrl_e = Metabolite(
        'tyrl_e',
        formula = 'C8H9O2',
        name = 'tyrosol',
        compartment = 'e')
    
    tyrl_c = Metabolite(
        'tyrl_c',
        formula = 'C8H9O2',
        name = 'tyrosol',
        compartment = 'c')
    
    htl_e = Metabolite(
        'htl_e',
        formula = 'C8H10O3',
        name = 'hydroxytyrosol',
        compartment = 'e')
    
    htl_c = Metabolite(
        'htl_c',
        formula = 'C8H10O3',
        name = 'hydroxytyrosol',
        compartment = 'c')
    
    ncam_e = Metabolite(
        'ncam_e',
        formula = 'C6H6N2O',
        name = 'Nicotinamide',
        compartment = 'e')

    model.add_metabolites(tyrl_e)
    model.add_metabolites(tyrl_c)
    model.add_metabolites(htl_e)
    model.add_metabolites(htl_c)
    model.add_metabolites(ncam_e)

    ## get the ids of relative metabolites in the metabolic network
    h_c = model.metabolites.get_by_id('h_c')
    nadh_c = model.metabolites.get_by_id('nadh_c')
    o2_c = model.metabolites.get_by_id('o2_c')
    nad_c = model.metabolites.get_by_id('nad_c')
    h2o_c = model.metabolites.get_by_id('h2o_c')
    ncam_c = model.metabolites.get_by_id('ncam_c')
    
    ## add new reactions
    model.add_reactions([Reaction('EX_tyrl_e_reverse')])
    model.reactions.EX_tyrl_e_reverse.add_metabolites({
        tyrl_e: 1.0,
    })
    
    model.add_reactions([Reaction('tyrl_e_e')])
    model.reactions.tyrl_e_e.add_metabolites({
        tyrl_e: -1.0,
        tyrl_c: 1.0,
    })
    
    model.add_reactions([Reaction('HpaBC')])
    model.reactions.HpaBC.add_metabolites({
        tyrl_c: -1.0,
        nadh_c: -1.0,
        h_c: -1.0,
        o2_c: -1.0,
        htl_c: 1.0,
        nad_c: 1.0,
        h2o_c: 1.0,
    })
    
    model.add_reactions([Reaction('htl_e_e')])
    model.reactions.htl_e_e.add_metabolites({
        htl_c: -1.0,
        htl_e: 1.0
    })
    
    model.add_reactions([Reaction('EX_htl_e')])
    model.reactions.EX_htl_e.add_metabolites({
        htl_e: -1.0,
    })
    
    model.add_reactions([Reaction('EX_ncam_e_reverse')])
    model.reactions.EX_ncam_e_reverse.add_metabolites({
        ncam_e: 1.0,
    })
    
    model.add_reactions([Reaction('ncam_e_e')])
    model.reactions.ncam_e_e.add_metabolites({
        ncam_e: -1.0,
        ncam_c: 1.0,
    })
    
    ## generate the metabolic network of the recombinant strain
    write_sbml_model(model,'../biosyntehsis-growth relationship/hydroxytyrosol_CCHT-2.xml')
    write_sbml_model(model,'../production prediction/hydroxytyrosol_CCHT-2.xml')
    write_sbml_model(model,'../metabolic engineering strategy optimization/hydroxytyrosol_CCHT-2.xml')